# PYNQ MD5 driver v Python

In [1]:
import numpy as np
from pynq import allocate, Overlay, Clocks
from pprint import pprint

# Define custom types
uint512 = np.dtype((np.uint8, 64))
uint128 = np.dtype((np.uint8, 16))

def print_uint512(arr):
    for i in range(len(arr)):
        data = int.from_bytes(arr[i], byteorder='big', signed=False)
        numString = f'{data:x}'.zfill(128)
        print(f'uint512[{i}] = {numString}')
        
def print_uint128(arr):
    for i in range(len(arr)):
        data = int.from_bytes(arr[i], byteorder='big', signed=False)
        numString = f'{data:x}'.zfill(32)
        print(f'uint128[{i}] = {numString}')

class MD5Overlay(Overlay):
    def __init__(self, bitfile, **kwargs):
        super().__init__(bitfile, **kwargs)
    
    def interleave(self, arrays:list): # TODO: je to ubistvu samo transpose?
        
        interleaved = []
        
        # First and last dimensions are always 64
        _, inputs_len, _ = np.shape(arrays)
                        
        for i in range(inputs_len):
            for j in range (64):
                interleaved.append(arrays[j][i])
        
        return interleaved
    
    def prepare_for_md5(self, input_string:bytearray):
    
        # Convert input string to bytearray
        #input_string = bytearray(input_string.encode())
        data_len = (8 * len(input_string)).to_bytes(8, byteorder='little')

        # Calculate the number of null bytes needed for padding
        padding_len = 64 - ((len(input_string) + 8) % 64)

        # Pad the input string with null bytes and the length of the original data
        input_string += b'\x80' + b'\x00' * (padding_len - 1)
        input_string += data_len

        data_blocks = np.reshape(input_string, (-1, 64))

        return data_blocks

    def prepare_input(self, inputs:list):
        
        # Razdeli vhode na seznam, ki vsebuje arraye 512-bit blokov
        inputs_prepared = []
        for inp in inputs:
            inputs_prepared.append(self.prepare_for_md5(inp))
                
        # Dodaj toliko inputov da jih bo skupaj 64 (dodani inputi so enake oblike kot prvi input)
        inputs_count, inputs_len, _ = np.shape(inputs_prepared)
        filler = np.zeros(shape=(64-inputs_count, inputs_len), dtype=uint512)
        inputs_prepared.extend(filler)
        
        # Pripravi vrstni red blokov za cevovod
        interleaved = self.interleave(inputs_prepared)
        
        input_buffer = allocate(shape=(len(interleaved),), dtype=uint512)
        np.copyto(input_buffer, interleaved)
        
        return input_buffer
    
    def md5(self, input_buffer, output_buffer):
    
        dma = self.axi_dma_0
        dma.sendchannel.transfer(input_buffer)
        dma.recvchannel.transfer(output_buffer)
        dma.sendchannel.wait()
        dma.recvchannel.wait()
        
        return output_buffer

overlay = MD5Overlay('/home/xilinx/pynq/overlays/pynq-MD5_2/pynq-MD5_2.bit')
Clocks.fclk0_mhz = 300
#data_list = [bytearray([i+97] * 64) for i in range(64)]
data_list = [bytearray(b'a' + b'\0'*63) for i in range(64)]

input_buffer = overlay.prepare_input(data_list)
output_buffer = allocate(shape=(64,), dtype=uint128)

/usr/local/share/pynq-venv/lib/python3.10/site-packages/pynq/ps.py:434: UserWarning: Setting frequency to the closest possible value 98.8879MHz.
  warnings.warn(
/usr/local/share/pynq-venv/lib/python3.10/site-packages/pynq/ps.py:434: UserWarning: Setting frequency to the closest possible value 296.6637MHz.
  warnings.warn(


In [2]:
#%%timeit -r 1 -n 1
input_buffer = overlay.prepare_input(data_list)
print_uint512(input_buffer)
#overlay.md5(input_buffer, output_buffer)
#print_uint128(hash)

uint512[0] = 61000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
uint512[1] = 61000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
uint512[2] = 61000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
uint512[3] = 61000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
uint512[4] = 61000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
uint512[5] = 61000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
uint512[6] = 61000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
uint51